In [1]:
# Test
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
#檢測tensorflow gpu是否啟用
tf.config.list_physical_devices("GPU")
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [3]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import os
os.environ["CUDA_DEVICE_ORDER"] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'#选择哪一块gpu
config = ConfigProto()
# config.allow_soft_placement=True #如果你指定的设备不存在，允许TF自动分配设备
# config.gpu_options.per_process_gpu_memory_fraction=0.5 #分配百分之七十的显存给程序使用，避免内存溢出，可以自己调整
config.gpu_options.allow_growth = True #按

In [4]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
input_layer = Input(shape=(216, 384,1))

B7 = tf.keras.applications.efficientnet.EfficientNetB7(include_top=False, weights=None,input_tensor = input_layer,classifier_activation='softmax')

In [5]:
B7.summary()

Model: "efficientnetb7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 216, 384, 1) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 216, 384, 1)  0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 216, 384, 1)  3           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 217, 385, 1)  0           normalization[0][0]              
_____________________________________________________________________________________

In [6]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input
from tensorflow.keras.layers.experimental.preprocessing import Rescaling , Normalization
from tensorflow.keras.layers import ZeroPadding2D , Conv2D , BatchNormalization , Activation , DepthwiseConv2D , GlobalAveragePooling2D
from tensorflow.keras.layers import Reshape , Multiply , Dropout , Add , ZeroPadding2D
from tensorflow.keras.models import Model

input_layer = Input(shape=(216,384,1) , name='input_layer')
rescaling = Rescaling(scale=1./255. , name='rescaling')(input_layer)
normalization = Normalization(axis=-1 , name='normalization')(rescaling)
stem_conv_pad = ZeroPadding2D(padding = (1,1) , data_format = 'channels_last' , name = 'stem_conv_pad')(normalization)
stem_conv = Conv2D(64,kernel_size = (3,3),activation='relu' , name='stem_conv' ,padding = 'valid' , strides=(2, 2))(stem_conv_pad)
stem_bn = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 , name='stem_bn')(stem_conv)
stem_activation = Activation('relu' , name='stem_activation')(stem_bn)

# Block1

In [7]:
# Block 1

## Sub Block1
block1a_dwconv = DepthwiseConv2D(kernel_size = (1,1),activation='relu' , padding = 'valid' , data_format='channels_last', name='block1a_dwconv' )(stem_activation)
block1a_bn = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 , name = 'block1a_bn')(block1a_dwconv)
block1a_activation = Activation('relu' , name='block1a_activation')(block1a_bn)
block1a_se_squeeze = GlobalAveragePooling2D(data_format='channels_last' , name = 'block1a_se_squeeze')(block1a_activation)
block1a_se_reshape = Reshape((1,1,-1) , name='block1a_se_reshape')(block1a_se_squeeze)
block1a_se_reduce = Conv2D(16 , kernel_size = (1,1) , activation = 'relu' , name='block1a_se_reduce' , padding = 'valid')(block1a_se_reshape)
block1a_se_expand = Conv2D(64 , kernel_size = (1,1) , activation = 'relu' , name='block1a_se_expand' , padding = 'valid')(block1a_se_reduce)
block1a_se_excite = Multiply(name = 'block1a_se_excite')([block1a_activation, block1a_se_expand])
block1a_project_conv = Conv2D(32,kernel_size=(1,1),data_format = 'channels_last' , padding='valid' , activation='relu' , strides = (1,1) , name = 'block1a_project_conv')(block1a_se_excite)
block1a_project_bn = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 , name = 'block1a_project_bn')(block1a_project_conv)

## Sub Block4
block1b_dwconv = DepthwiseConv2D(kernel_size = (1,1),activation='relu' , padding = 'valid' , data_format='channels_last', name='block1b_dwconv' )(block1a_project_bn)
block1b_bn = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 , name = 'block1b_bn')(block1b_dwconv)
block1b_activation = Activation('relu' , name='block1b_activation')(block1b_bn)
block1b_se_squeeze = GlobalAveragePooling2D(data_format='channels_last' , name = 'block1b_se_squeeze')(block1b_activation)
block1b_se_reshape = Reshape((1,1,-1) , name='block1b_se_reshape')(block1b_se_squeeze)
block1b_se_reduce = Conv2D(8 , kernel_size = (1,1) , activation = 'relu' , name='block1b_se_reduce' , padding = 'valid')(block1b_se_reshape)
block1b_se_expand = Conv2D(32 , kernel_size = (1,1) , activation = 'relu' , name='block1b_se_expand' , padding = 'valid')(block1b_se_reduce)
block1b_se_excite = Multiply(name = 'block1b_se_excite')([block1b_activation , block1b_se_expand])
block1b_project_conv = Conv2D(32,kernel_size=(1,1),data_format = 'channels_last' , padding='valid' , activation='relu' , strides = (1,1) , name = 'block1b_project_conv')(block1b_se_excite)
block1b_project_bn = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 , name = 'block1b_project_bn')(block1b_project_conv)
block1b_drop = Dropout(0.2 , name='block1b_drop')(block1b_project_bn)
block1b_add = Add(name='block1b_add')([block1b_drop , block1a_project_bn])

## Sub block4
block1c_dwconv = DepthwiseConv2D(kernel_size = (1,1),activation='relu' , padding = 'valid' , data_format='channels_last', name='block1c_dwconv' )(block1b_add)
block1c_bn = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 , name = 'block1c_bn')(block1c_dwconv)
block1c_activation = Activation('relu' , name='block1c_activation')(block1c_bn)
block1c_se_squeeze = GlobalAveragePooling2D(data_format='channels_last' , name = 'block1c_se_squeeze')(block1c_activation)
block1c_se_reshape = Reshape((1,1,-1) , name='block1c_se_reshape')(block1c_se_squeeze)
block1c_se_reduce = Conv2D(8 , kernel_size = (1,1) , activation = 'relu' , name='block1c_se_reduce' , padding = 'valid')(block1c_se_reshape)
block1c_se_expand = Conv2D(32 , kernel_size = (1,1) , activation = 'relu' , name='block1c_se_expand' , padding = 'valid')(block1c_se_reduce)
block1c_se_excite = Multiply(name = 'block1c_se_excite')([block1c_activation , block1c_se_expand])
block1c_project_conv = Conv2D(32,kernel_size=(1,1),data_format = 'channels_last' , padding='valid' , activation='relu' , strides = (1,1) , name = 'block1c_project_conv')(block1c_se_excite)
block1c_project_bn = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 , name = 'block1c_project_bn')(block1c_project_conv)
block1c_drop = Dropout(0.2 , name='block1c_drop')(block1c_project_bn)
block1c_add = Add(name='block1c_add')([block1c_drop , block1b_add])

# Block2

In [8]:
# Block 2
## Sub Block2
block2a_expand_conv = Conv2D(192,kernel_size=(1,1),data_format = 'channels_last' , padding='valid' , activation='relu' , strides = (1,1) , name = 'block2a_expand_conv')(block1c_add)
block2a_expand_bn = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 , name = 'block2a_expand_bn')(block2a_expand_conv)
block2a_expand_activation = Activation('relu' , name='block2a_expand_activation')(block2a_expand_bn)
block2a_dwconv_pad = ZeroPadding2D(padding=1 , data_format = 'channels_last' , name = 'block2a_dwconv_pad')(block2a_expand_activation)
block2a_dwconv = DepthwiseConv2D(kernel_size = 3,activation='relu' , padding = 'valid' , data_format='channels_last', strides = (2,2) ,name='block2a_dwconv' )(block2a_dwconv_pad)
block2a_bn = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 , name = 'block2a_bn')(block2a_dwconv)
block2a_activation = Activation('relu' , name='block2a_activation')(block2a_bn)
block2a_se_squeeze = GlobalAveragePooling2D(data_format='channels_last' , name = 'block2a_se_squeeze')(block2a_activation)
block2a_se_reshape = Reshape((1,1,-1) , name='block2a_se_reshape')(block2a_se_squeeze)
block2a_se_reduce = Conv2D(8 , kernel_size = (1,1) , activation ='relu', padding='valid', name = 'block2a_se_reduce')(block2a_se_reshape)
block2a_se_expand = Conv2D(192 , kernel_size = (1,1) , activation ='relu', padding='valid', name = 'block2a_se_expand')(block2a_se_reduce)
block2a_se_excite = Multiply(name='block2a_se_excite')([block2a_activation,block2a_se_expand])
block2a_project_conv = Conv2D(48 , kernel_size=(1,1) , activation ='relu', padding='valid', name='block2a_project_conv')(block2a_se_excite)
block2a_project_bn = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 , name = 'block2a_project_bn')(block2a_project_conv)


## Sub Block3
block2b_expand_conv = Conv2D(288,kernel_size=(1,1),data_format='channels_last',padding='valid',activation='relu',strides=(1,1),name='block2b_expand_conv')(block2a_project_bn)
block2b_expand_bn = BatchNormalization(axis=-1,momentum=0.99,epsilon=0.001,name='block2b_expand_bn')(block2b_expand_conv)
block2b_expand_activation = Activation('relu',name='block2b_expand_activation')(block2b_expand_bn)
block2b_dwconv = DepthwiseConv2D(kernel_size=1,activation='relu',padding='valid',data_format='channels_last',name='block2b_dwconv')(block2b_expand_activation)
block2b_bn = BatchNormalization(axis=-1,momentum=0.99,epsilon=0.001,name='block2b_bn')(block2b_dwconv)
block2b_activation = Activation('relu',name='block2b_activation')(block2b_bn)
block2b_se_squeeze = GlobalAveragePooling2D(data_format='channels_last',name='block2b_se_squeeze')(block2b_activation)
block2b_se_reshape = Reshape((1,1,-1),name='block2b_se_reshape')(block2b_se_squeeze)
block2b_se_reduce = Conv2D(12,kernel_size=(1,1),activation='relu',padding='valid',data_format='channels_last',name='block2b_se_reduce')(block2b_se_reshape)
block2b_se_expand = Conv2D(288,kernel_size=(1,1),activation='relu',padding='valid',data_format='channels_last',name='block2b_se_expand')(block2b_se_reduce)
block2b_se_excite = Multiply(name='block2b_se_excite')([block2b_activation,block2b_se_expand])
block2b_project_conv = Conv2D(48,kernel_size=(1,1),activation='relu',padding='valid',data_format='channels_last',name='block2b_project_conv')(block2b_se_excite)
block2b_project_bn = BatchNormalization(axis=-1,momentum=0.99,epsilon=0.001,name='block2b_project_bn')(block2b_project_conv)
block2b_drop = Dropout(0.2 , name='block2b_drop')(block2b_project_bn)
block2b_add = Add(name='block2b_add')([block2b_drop,block2a_project_bn])


## Sub Block3
block2c_expand_conv = Conv2D(288,kernel_size=(1,1),data_format='channels_last',padding='valid',activation='relu',strides=(1,1),name='block2c_expand_conv')(block2b_project_bn)
block2c_expand_bn = BatchNormalization(axis=-1,momentum=0.99,epsilon=0.001,name='block2c_expand_bn')(block2c_expand_conv)
block2c_expand_activation = Activation('relu',name='block2c_expand_activation')(block2c_expand_bn)
block2c_dwconv = DepthwiseConv2D(kernel_size=1,activation='relu',padding='valid',data_format='channels_last',name='block2c_dwconv')(block2c_expand_activation)
block2c_bn = BatchNormalization(axis=-1,momentum=0.99,epsilon=0.001,name='block2c_bn')(block2c_dwconv)
block2c_activation = Activation('relu',name='block2c_activation')(block2c_bn)
block2c_se_squeeze = GlobalAveragePooling2D(data_format='channels_last',name='block2c_se_squeeze')(block2c_activation)
block2c_se_reshape = Reshape((1,1,-1),name='block2c_se_reshape')(block2c_se_squeeze)
block2c_se_reduce = Conv2D(12,kernel_size=(1,1),activation='relu',padding='valid',data_format='channels_last',name='block2c_se_reduce')(block2c_se_reshape)
block2c_se_expand = Conv2D(288,kernel_size=(1,1),activation='relu',padding='valid',data_format='channels_last',name='block2c_se_expand')(block2c_se_reduce)
block2c_se_excite = Multiply(name='block2c_se_excite')([block2c_activation,block2c_se_expand])
block2c_project_conv = Conv2D(48,kernel_size=(1,1),activation='relu',padding='valid',data_format='channels_last',name='block2c_project_conv')(block2c_se_excite)
block2c_project_bn = BatchNormalization(axis=-1,momentum=0.99,epsilon=0.001,name='block2c_project_bn')(block2c_project_conv)
block2c_drop = Dropout(0.2 , name='block2c_drop')(block2c_project_bn)
block2c_add = Add(name='block2b_add')([block2c_drop,block2b_project_bn])

# Block3

In [9]:
# Block3
## Sub Block2
block3a_expand_conv = Conv2D(288,kernel_size=(1,1),data_format = 'channels_last',padding='valid',activation='relu',strides = (1,1),name = 'block3a_expand_conv')(block2c_add)
block3a_expand_bn = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 , name = 'block3a_expand_bn')(block3a_expand_conv)
block3a_expand_activation = Activation('relu' , name='block3a_expand_activation')(block3a_expand_bn)
block3a_dwconv_pad = ZeroPadding2D(padding=1 , data_format = 'channels_last' , name = 'block3a_dwconv_pad')(block3a_expand_activation)
block3a_dwconv = DepthwiseConv2D(kernel_size = 3,activation='relu' , padding = 'valid' , data_format='channels_last', strides = (2,2) ,name='block3a_dwconv' )(block3a_dwconv_pad)
block3a_bn = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 , name = 'block3a_bn')(block3a_dwconv)
block3a_activation = Activation('relu' , name='block3a_activation')(block3a_bn)
block3a_se_squeeze = GlobalAveragePooling2D(data_format='channels_last' , name = 'block3a_se_squeeze')(block3a_activation)
block3a_se_reshape = Reshape((1,1,-1) , name='block3a_se_reshape')(block3a_se_squeeze)
block3a_se_reduce = Conv2D(12 , kernel_size = (1,1) , activation ='relu', padding='valid', name = 'block3a_se_reduce')(block3a_se_reshape)
block3a_se_expand = Conv2D(288 , kernel_size = (1,1) , activation ='relu', padding='valid', name = 'block3a_se_expand')(block3a_se_reduce)
block3a_se_excite = Multiply(name='block3a_se_excite')([block3a_activation,block3a_se_expand])
block3a_project_conv = Conv2D(80 , kernel_size=(1,1) , activation ='relu', padding='valid', name='block3a_project_conv')(block3a_se_excite)
block3a_project_bn = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 , name = 'block3a_project_bn')(block3a_project_conv)


## Sub Block3
block3b_expand_conv = Conv2D(480,kernel_size=(1,1),data_format='channels_last',padding='valid',activation='relu',strides=(1,1),name='block3b_expand_conv')(block3a_project_bn)
block3b_expand_bn = BatchNormalization(axis=-1,momentum=0.99,epsilon=0.001,name='block3b_expand_bn')(block3b_expand_conv)
block3b_expand_activation = Activation('relu',name='block3b_expand_activation')(block3b_expand_bn)
block3b_dwconv = DepthwiseConv2D(kernel_size=1,activation='relu',padding='valid',data_format='channels_last',name='block3b_dwconv')(block3b_expand_activation)
block3b_bn = BatchNormalization(axis=-1,momentum=0.99,epsilon=0.001,name='block3b_bn')(block3b_dwconv)
block3b_activation = Activation('relu',name='block3b_activation')(block3b_bn)
block3b_se_squeeze = GlobalAveragePooling2D(data_format='channels_last',name='block3b_se_squeeze')(block3b_activation)
block3b_se_reshape = Reshape((1,1,-1),name='block3b_se_reshape')(block3b_se_squeeze)
block3b_se_reduce = Conv2D(20,kernel_size=(1,1),activation='relu',padding='valid',data_format='channels_last',name='block3b_se_reduce')(block3b_se_reshape)
block3b_se_expand = Conv2D(480,kernel_size=(1,1),activation='relu',padding='valid',data_format='channels_last',name='block3b_se_expand')(block3b_se_reduce)
block3b_se_excite = Multiply(name='block3b_se_excite')([block3b_activation,block3b_se_expand])
block3b_project_conv = Conv2D(80,kernel_size=(1,1),activation='relu',padding='valid',data_format='channels_last',name='block3b_project_conv')(block3b_se_excite)
block3b_project_bn = BatchNormalization(axis=-1,momentum=0.99,epsilon=0.001,name='block3b_project_bn')(block3b_project_conv)
block3b_drop = Dropout(0.2 , name='block3b_drop')(block3b_project_bn)
block3b_add = Add(name='block3b_add')([block3b_drop,block3a_project_bn])


## Sub Block3
block3c_expand_conv = Conv2D(480,kernel_size=(1,1),data_format='channels_last',padding='valid',activation='relu',strides=(1,1),name='block3c_expand_conv')(block3b_project_bn)
block3c_expand_bn = BatchNormalization(axis=-1,momentum=0.99,epsilon=0.001,name='block3c_expand_bn')(block3c_expand_conv)
block3c_expand_activation = Activation('relu',name='block3c_expand_activation')(block3c_expand_bn)
block3c_dwconv = DepthwiseConv2D(kernel_size=1,activation='relu',padding='valid',data_format='channels_last',name='block3c_dwconv')(block3c_expand_activation)
block3c_bn = BatchNormalization(axis=-1,momentum=0.99,epsilon=0.001,name='block3c_bn')(block3c_dwconv)
block3c_activation = Activation('relu',name='block3c_activation')(block3c_bn)
block3c_se_squeeze = GlobalAveragePooling2D(data_format='channels_last',name='block3c_se_squeeze')(block3c_activation)
block3c_se_reshape = Reshape((1,1,-1),name='block3c_se_reshape')(block3c_se_squeeze)
block3c_se_reduce = Conv2D(20,kernel_size=(1,1),activation='relu',padding='valid',data_format='channels_last',name='block3c_se_reduce')(block3c_se_reshape)
block3c_se_expand = Conv2D(480,kernel_size=(1,1),activation='relu',padding='valid',data_format='channels_last',name='block3c_se_expand')(block3c_se_reduce)
block3c_se_excite = Multiply(name='block3c_se_excite')([block3c_activation,block3c_se_expand])
block3c_project_conv = Conv2D(80,kernel_size=(1,1),activation='relu',padding='valid',data_format='channels_last',name='block3c_project_conv')(block3c_se_excite)
block3c_project_bn = BatchNormalization(axis=-1,momentum=0.99,epsilon=0.001,name='block3c_project_bn')(block3c_project_conv)
block3c_drop = Dropout(0.2 , name='block3c_drop')(block3c_project_bn)
block3c_add = Add(name='block3b_add')([block3c_drop,block3b_project_bn])

# Block4

In [10]:
# Block4
## Sub Block2
block4a_expand_conv = Conv2D(480,kernel_size=(1,1),data_format = 'channels_last' , padding='valid' , activation='relu' , strides = (1,1) , name = 'block4a_expand_conv')(block3c_add)
block4a_expand_bn = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 , name = 'block4a_expand_bn')(block4a_expand_conv)
block4a_expand_activation = Activation('relu' , name='block4a_expand_activation')(block4a_expand_bn)
block4a_dwconv_pad = ZeroPadding2D(padding=1 , data_format = 'channels_last' , name = 'block4a_dwconv_pad')(block4a_expand_activation)
block4a_dwconv = DepthwiseConv2D(kernel_size = 3,activation='relu' , padding = 'valid' , data_format='channels_last', strides = (2,2) ,name='block4a_dwconv' )(block4a_dwconv_pad)
block4a_bn = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 , name = 'block4a_bn')(block4a_dwconv)
block4a_activation = Activation('relu' , name='block4a_activation')(block4a_bn)
block4a_se_squeeze = GlobalAveragePooling2D(data_format='channels_last' , name = 'block4a_se_squeeze')(block4a_activation)
block4a_se_reshape = Reshape((1,1,-1) , name='block4a_se_reshape')(block4a_se_squeeze)
block4a_se_reduce = Conv2D(20 , kernel_size = (1,1) , activation ='relu', padding='valid', name = 'block4a_se_reduce')(block4a_se_reshape)
block4a_se_expand = Conv2D(480 , kernel_size = (1,1) , activation ='relu', padding='valid', name = 'block4a_se_expand')(block4a_se_reduce)
block4a_se_excite = Multiply(name='block4a_se_excite')([block4a_activation,block4a_se_expand])
block4a_project_conv = Conv2D(160 , kernel_size=(1,1) , activation ='relu', padding='valid', name='block4a_project_conv')(block4a_se_excite)
block4a_project_bn = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 , name = 'block4a_project_bn')(block4a_project_conv)


## Sub Block3
block4b_expand_conv = Conv2D(960,kernel_size=(1,1),data_format='channels_last',padding='valid',activation='relu',strides=(1,1),name='block4b_expand_conv')(block4a_project_bn)
block4b_expand_bn = BatchNormalization(axis=-1,momentum=0.99,epsilon=0.001,name='block4b_expand_bn')(block4b_expand_conv)
block4b_expand_activation = Activation('relu',name='block4b_expand_activation')(block4b_expand_bn)
block4b_dwconv = DepthwiseConv2D(kernel_size=1,activation='relu',padding='valid',data_format='channels_last',name='block4b_dwconv')(block4b_expand_activation)
block4b_bn = BatchNormalization(axis=-1,momentum=0.99,epsilon=0.001,name='block4b_bn')(block4b_dwconv)
block4b_activation = Activation('relu',name='block4b_activation')(block4b_bn)
block4b_se_squeeze = GlobalAveragePooling2D(data_format='channels_last',name='block4b_se_squeeze')(block4b_activation)
block4b_se_reshape = Reshape((1,1,-1),name='block4b_se_reshape')(block4b_se_squeeze)
block4b_se_reduce = Conv2D(40,kernel_size=(1,1),activation='relu',padding='valid',data_format='channels_last',name='block4b_se_reduce')(block4b_se_reshape)
block4b_se_expand = Conv2D(960,kernel_size=(1,1),activation='relu',padding='valid',data_format='channels_last',name='block4b_se_expand')(block4b_se_reduce)
block4b_se_excite = Multiply(name='block4b_se_excite')([block4b_activation,block4b_se_expand])
block4b_project_conv = Conv2D(160,kernel_size=(1,1),activation='relu',padding='valid',data_format='channels_last',name='block4b_project_conv')(block4b_se_excite)
block4b_project_bn = BatchNormalization(axis=-1,momentum=0.99,epsilon=0.001,name='block4b_project_bn')(block4b_project_conv)
block4b_drop = Dropout(0.2 , name='block4b_drop')(block4b_project_bn)
block4b_add = Add(name='block4b_add')([block4b_drop,block4a_project_bn])


## Sub Block3
block4c_expand_conv = Conv2D(960,kernel_size=(1,1),data_format='channels_last',padding='valid',activation='relu',strides=(1,1),name='block4c_expand_conv')(block4b_project_bn)
block4c_expand_bn = BatchNormalization(axis=-1,momentum=0.99,epsilon=0.001,name='block4c_expand_bn')(block4c_expand_conv)
block4c_expand_activation = Activation('relu',name='block4c_expand_activation')(block4c_expand_bn)
block4c_dwconv = DepthwiseConv2D(kernel_size=1,activation='relu',padding='valid',data_format='channels_last',name='block4c_dwconv')(block4c_expand_activation)
block4c_bn = BatchNormalization(axis=-1,momentum=0.99,epsilon=0.001,name='block4c_bn')(block4c_dwconv)
block4c_activation = Activation('relu',name='block4c_activation')(block4c_bn)
block4c_se_squeeze = GlobalAveragePooling2D(data_format='channels_last',name='block4c_se_squeeze')(block4c_activation)
block4c_se_reshape = Reshape((1,1,-1),name='block4c_se_reshape')(block4c_se_squeeze)
block4c_se_reduce = Conv2D(40,kernel_size=(1,1),activation='relu',padding='valid',data_format='channels_last',name='block4c_se_reduce')(block4c_se_reshape)
block4c_se_expand = Conv2D(960,kernel_size=(1,1),activation='relu',padding='valid',data_format='channels_last',name='block4c_se_expand')(block4c_se_reduce)
block4c_se_excite = Multiply(name='block4c_se_excite')([block4c_activation,block4c_se_expand])
block4c_project_conv = Conv2D(160,kernel_size=(1,1),activation='relu',padding='valid',data_format='channels_last',name='block4c_project_conv')(block4c_se_excite)
block4c_project_bn = BatchNormalization(axis=-1,momentum=0.99,epsilon=0.001,name='block4c_project_bn')(block4c_project_conv)
block4c_drop = Dropout(0.2 , name='block4c_drop')(block4c_project_bn)
block4c_add = Add(name='block4b_add')([block4c_drop,block4b_project_bn])

# Block5

In [11]:
# Block5
## Sub Block2
block5a_expand_conv = Conv2D(960,kernel_size=(1,1),data_format = 'channels_last' , padding='valid' , activation='relu' , strides = (1,1) , name = 'block5a_expand_conv')(block4c_add)
block5a_expand_bn = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 , name = 'block5a_expand_bn')(block5a_expand_conv)
block5a_expand_activation = Activation('relu' , name='block5a_expand_activation')(block5a_expand_bn)
block5a_dwconv_pad = ZeroPadding2D(padding=1 , data_format = 'channels_last' , name = 'block5a_dwconv_pad')(block5a_expand_activation)
block5a_dwconv = DepthwiseConv2D(kernel_size = 3,activation='relu' , padding = 'valid' , data_format='channels_last', strides = (2,2) ,name='block5a_dwconv' )(block5a_dwconv_pad)
block5a_bn = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 , name = 'block5a_bn')(block5a_dwconv)
block5a_activation = Activation('relu' , name='block5a_activation')(block5a_bn)
block5a_se_squeeze = GlobalAveragePooling2D(data_format='channels_last' , name = 'block5a_se_squeeze')(block5a_activation)
block5a_se_reshape = Reshape((1,1,-1) , name='block5a_se_reshape')(block5a_se_squeeze)
block5a_se_reduce = Conv2D(40 , kernel_size = (1,1) , activation ='relu', padding='valid', name = 'block5a_se_reduce')(block5a_se_reshape)
block5a_se_expand = Conv2D(960 , kernel_size = (1,1) , activation ='relu', padding='valid', name = 'block5a_se_expand')(block5a_se_reduce)
block5a_se_excite = Multiply(name='block5a_se_excite')([block5a_activation,block5a_se_expand])
block5a_project_conv = Conv2D(224 , kernel_size=(1,1) , activation ='relu', padding='valid', name='block5a_project_conv')(block5a_se_excite)
block5a_project_bn = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 , name = 'block5a_project_bn')(block5a_project_conv)


## Sub Block3
block5b_expand_conv = Conv2D(1344,kernel_size=(1,1),data_format='channels_last',padding='valid',activation='relu',strides=(1,1),name='block5b_expand_conv')(block5a_project_bn)
block5b_expand_bn = BatchNormalization(axis=-1,momentum=0.99,epsilon=0.001,name='block5b_expand_bn')(block5b_expand_conv)
block5b_expand_activation = Activation('relu',name='block5b_expand_activation')(block5b_expand_bn)
block5b_dwconv = DepthwiseConv2D(kernel_size=1,activation='relu',padding='valid',data_format='channels_last',name='block5b_dwconv')(block5b_expand_activation)
block5b_bn = BatchNormalization(axis=-1,momentum=0.99,epsilon=0.001,name='block5b_bn')(block5b_dwconv)
block5b_activation = Activation('relu',name='block5b_activation')(block5b_bn)
block5b_se_squeeze = GlobalAveragePooling2D(data_format='channels_last',name='block5b_se_squeeze')(block5b_activation)
block5b_se_reshape = Reshape((1,1,-1),name='block5b_se_reshape')(block5b_se_squeeze)
block5b_se_reduce = Conv2D(56,kernel_size=(1,1),activation='relu',padding='valid',data_format='channels_last',name='block5b_se_reduce')(block5b_se_reshape)
block5b_se_expand = Conv2D(1344,kernel_size=(1,1),activation='relu',padding='valid',data_format='channels_last',name='block5b_se_expand')(block5b_se_reduce)
block5b_se_excite = Multiply(name='block5b_se_excite')([block5b_activation,block5b_se_expand])
block5b_project_conv = Conv2D(224,kernel_size=(1,1),activation='relu',padding='valid',data_format='channels_last',name='block5b_project_conv')(block5b_se_excite)
block5b_project_bn = BatchNormalization(axis=-1,momentum=0.99,epsilon=0.001,name='block5b_project_bn')(block5b_project_conv)
block5b_drop = Dropout(0.2 , name='block5b_drop')(block5b_project_bn)
block5b_add = Add(name='block5b_add')([block5b_drop,block5a_project_bn])


## Sub Block3
block5c_expand_conv = Conv2D(1344,kernel_size=(1,1),data_format='channels_last',padding='valid',activation='relu',strides=(1,1),name='block5c_expand_conv')(block5b_project_bn)
block5c_expand_bn = BatchNormalization(axis=-1,momentum=0.99,epsilon=0.001,name='block5c_expand_bn')(block5c_expand_conv)
block5c_expand_activation = Activation('relu',name='block5c_expand_activation')(block5c_expand_bn)
block5c_dwconv = DepthwiseConv2D(kernel_size=1,activation='relu',padding='valid',data_format='channels_last',name='block5c_dwconv')(block5c_expand_activation)
block5c_bn = BatchNormalization(axis=-1,momentum=0.99,epsilon=0.001,name='block5c_bn')(block5c_dwconv)
block5c_activation = Activation('relu',name='block5c_activation')(block5c_bn)
block5c_se_squeeze = GlobalAveragePooling2D(data_format='channels_last',name='block5c_se_squeeze')(block5c_activation)
block5c_se_reshape = Reshape((1,1,-1),name='block5c_se_reshape')(block5c_se_squeeze)
block5c_se_reduce = Conv2D(56,kernel_size=(1,1),activation='relu',padding='valid',data_format='channels_last',name='block5c_se_reduce')(block5c_se_reshape)
block5c_se_expand = Conv2D(1344,kernel_size=(1,1),activation='relu',padding='valid',data_format='channels_last',name='block5c_se_expand')(block5c_se_reduce)
block5c_se_excite = Multiply(name='block5c_se_excite')([block5c_activation,block5c_se_expand])
block5c_project_conv = Conv2D(224,kernel_size=(1,1),activation='relu',padding='valid',data_format='channels_last',name='block5c_project_conv')(block5c_se_excite)
block5c_project_bn = BatchNormalization(axis=-1,momentum=0.99,epsilon=0.001,name='block5c_project_bn')(block5c_project_conv)
block5c_drop = Dropout(0.2 , name='block5c_drop')(block5c_project_bn)
block5c_add = Add(name='block5b_add')([block5c_drop,block5b_project_bn])

# Block6

In [12]:
# Block6
## Sub Block2
block6a_expand_conv = Conv2D(1344,kernel_size=(1,1),data_format = 'channels_last' , padding='valid' , activation='relu' , strides = (1,1) , name = 'block6a_expand_conv')(block5c_add)
block6a_expand_bn = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 , name = 'block6a_expand_bn')(block6a_expand_conv)
block6a_expand_activation = Activation('relu' , name='block6a_expand_activation')(block6a_expand_bn)
block6a_dwconv_pad = ZeroPadding2D(padding=1 , data_format = 'channels_last' , name = 'block6a_dwconv_pad')(block6a_expand_activation)
block6a_dwconv = DepthwiseConv2D(kernel_size = 3,activation='relu' , padding = 'valid' , data_format='channels_last', strides = (2,2) ,name='block6a_dwconv' )(block6a_dwconv_pad)
block6a_bn = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 , name = 'block6a_bn')(block6a_dwconv)
block6a_activation = Activation('relu' , name='block6a_activation')(block6a_bn)
block6a_se_squeeze = GlobalAveragePooling2D(data_format='channels_last' , name = 'block6a_se_squeeze')(block6a_activation)
block6a_se_reshape = Reshape((1,1,-1) , name='block6a_se_reshape')(block6a_se_squeeze)
block6a_se_reduce = Conv2D(56 , kernel_size = (1,1) , activation ='relu', padding='valid', name = 'block6a_se_reduce')(block6a_se_reshape)
block6a_se_expand = Conv2D(1344 , kernel_size = (1,1) , activation ='relu', padding='valid', name = 'block6a_se_expand')(block6a_se_reduce)
block6a_se_excite = Multiply(name='block6a_se_excite')([block6a_activation,block6a_se_expand])
block6a_project_conv = Conv2D(384 , kernel_size=(1,1) , activation ='relu', padding='valid', name='block6a_project_conv')(block6a_se_excite)
block6a_project_bn = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 , name = 'block6a_project_bn')(block6a_project_conv)


## Sub Block3
block6b_expand_conv = Conv2D(2304,kernel_size=(1,1),data_format='channels_last',padding='valid',activation='relu',strides=(1,1),name='block6b_expand_conv')(block6a_project_bn)
block6b_expand_bn = BatchNormalization(axis=-1,momentum=0.99,epsilon=0.001,name='block6b_expand_bn')(block6b_expand_conv)
block6b_expand_activation = Activation('relu',name='block6b_expand_activation')(block6b_expand_bn)
block6b_dwconv = DepthwiseConv2D(kernel_size=1,activation='relu',padding='valid',data_format='channels_last',name='block6b_dwconv')(block6b_expand_activation)
block6b_bn = BatchNormalization(axis=-1,momentum=0.99,epsilon=0.001,name='block6b_bn')(block6b_dwconv)
block6b_activation = Activation('relu',name='block6b_activation')(block6b_bn)
block6b_se_squeeze = GlobalAveragePooling2D(data_format='channels_last',name='block6b_se_squeeze')(block6b_activation)
block6b_se_reshape = Reshape((1,1,-1),name='block6b_se_reshape')(block6b_se_squeeze)
block6b_se_reduce = Conv2D(96,kernel_size=(1,1),activation='relu',padding='valid',data_format='channels_last',name='block6b_se_reduce')(block6b_se_reshape)
block6b_se_expand = Conv2D(2304,kernel_size=(1,1),activation='relu',padding='valid',data_format='channels_last',name='block6b_se_expand')(block6b_se_reduce)
block6b_se_excite = Multiply(name='block6b_se_excite')([block6b_activation,block6b_se_expand])
block6b_project_conv = Conv2D(384,kernel_size=(1,1),activation='relu',padding='valid',data_format='channels_last',name='block6b_project_conv')(block6b_se_excite)
block6b_project_bn = BatchNormalization(axis=-1,momentum=0.99,epsilon=0.001,name='block6b_project_bn')(block6b_project_conv)
block6b_drop = Dropout(0.2 , name='block6b_drop')(block6b_project_bn)
block6b_add = Add(name='block6b_add')([block6b_drop,block6a_project_bn])


## Sub Block3
block6c_expand_conv = Conv2D(2304,kernel_size=(1,1),data_format='channels_last',padding='valid',activation='relu',strides=(1,1),name='block6c_expand_conv')(block6b_project_bn)
block6c_expand_bn = BatchNormalization(axis=-1,momentum=0.99,epsilon=0.001,name='block6c_expand_bn')(block6c_expand_conv)
block6c_expand_activation = Activation('relu',name='block6c_expand_activation')(block6c_expand_bn)
block6c_dwconv = DepthwiseConv2D(kernel_size=1,activation='relu',padding='valid',data_format='channels_last',name='block6c_dwconv')(block6c_expand_activation)
block6c_bn = BatchNormalization(axis=-1,momentum=0.99,epsilon=0.001,name='block6c_bn')(block6c_dwconv)
block6c_activation = Activation('relu',name='block6c_activation')(block6c_bn)
block6c_se_squeeze = GlobalAveragePooling2D(data_format='channels_last',name='block6c_se_squeeze')(block6c_activation)
block6c_se_reshape = Reshape((1,1,-1),name='block6c_se_reshape')(block6c_se_squeeze)
block6c_se_reduce = Conv2D(96,kernel_size=(1,1),activation='relu',padding='valid',data_format='channels_last',name='block6c_se_reduce')(block6c_se_reshape)
block6c_se_expand = Conv2D(2304,kernel_size=(1,1),activation='relu',padding='valid',data_format='channels_last',name='block6c_se_expand')(block6c_se_reduce)
block6c_se_excite = Multiply(name='block6c_se_excite')([block6c_activation,block6c_se_expand])
block6c_project_conv = Conv2D(384,kernel_size=(1,1),activation='relu',padding='valid',data_format='channels_last',name='block6c_project_conv')(block6c_se_excite)
block6c_project_bn = BatchNormalization(axis=-1,momentum=0.99,epsilon=0.001,name='block6c_project_bn')(block6c_project_conv)
block6c_drop = Dropout(0.2 , name='block6c_drop')(block6c_project_bn)
block6c_add = Add(name='block6b_add')([block6c_drop,block6b_project_bn])

# Block7

In [13]:
# Block7
## Sub Block2
block7a_expand_conv = Conv2D(2304,kernel_size=(1,1),data_format = 'channels_last' , padding='valid' , activation='relu' , strides = (1,1) , name = 'block7a_expand_conv')(block6c_add)
block7a_expand_bn = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 , name = 'block7a_expand_bn')(block7a_expand_conv)
block7a_expand_activation = Activation('relu' , name='block7a_expand_activation')(block7a_expand_bn)
block7a_dwconv_pad = ZeroPadding2D(padding=1 , data_format = 'channels_last' , name = 'block7a_dwconv_pad')(block7a_expand_activation)
block7a_dwconv = DepthwiseConv2D(kernel_size = 3,activation='relu' , padding = 'valid' , data_format='channels_last', strides = (2,2) ,name='block7a_dwconv' )(block7a_dwconv_pad)
block7a_bn = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 , name = 'block7a_bn')(block7a_dwconv)
block7a_activation = Activation('relu' , name='block7a_activation')(block7a_bn)
block7a_se_squeeze = GlobalAveragePooling2D(data_format='channels_last' , name = 'block7a_se_squeeze')(block7a_activation)
block7a_se_reshape = Reshape((1,1,-1) , name='block7a_se_reshape')(block7a_se_squeeze)
block7a_se_reduce = Conv2D(96 , kernel_size = (1,1) , activation ='relu', padding='valid', name = 'block7a_se_reduce')(block7a_se_reshape)
block7a_se_expand = Conv2D(2304 , kernel_size = (1,1) , activation ='relu', padding='valid', name = 'block7a_se_expand')(block7a_se_reduce)
block7a_se_excite = Multiply(name='block7a_se_excite')([block7a_activation,block7a_se_expand])
block7a_project_conv = Conv2D(640 , kernel_size=(1,1) , activation ='relu', padding='valid', name='block7a_project_conv')(block7a_se_excite)
block7a_project_bn = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 , name = 'block7a_project_bn')(block7a_project_conv)


## Sub Block3
block7b_expand_conv = Conv2D(3840,kernel_size=(1,1),data_format='channels_last',padding='valid',activation='relu',strides=(1,1),name='block7b_expand_conv')(block7a_project_bn)
block7b_expand_bn = BatchNormalization(axis=-1,momentum=0.99,epsilon=0.001,name='block7b_expand_bn')(block7b_expand_conv)
block7b_expand_activation = Activation('relu',name='block7b_expand_activation')(block7b_expand_bn)
block7b_dwconv = DepthwiseConv2D(kernel_size=1,activation='relu',padding='valid',data_format='channels_last',name='block7b_dwconv')(block7b_expand_activation)
block7b_bn = BatchNormalization(axis=-1,momentum=0.99,epsilon=0.001,name='block7b_bn')(block7b_dwconv)
block7b_activation = Activation('relu',name='block7b_activation')(block7b_bn)
block7b_se_squeeze = GlobalAveragePooling2D(data_format='channels_last',name='block7b_se_squeeze')(block7b_activation)
block7b_se_reshape = Reshape((1,1,-1),name='block7b_se_reshape')(block7b_se_squeeze)
block7b_se_reduce = Conv2D(160,kernel_size=(1,1),activation='relu',padding='valid',data_format='channels_last',name='block7b_se_reduce')(block7b_se_reshape)
block7b_se_expand = Conv2D(3840,kernel_size=(1,1),activation='relu',padding='valid',data_format='channels_last',name='block7b_se_expand')(block7b_se_reduce)
block7b_se_excite = Multiply(name='block7b_se_excite')([block7b_activation,block7b_se_expand])
block7b_project_conv = Conv2D(640,kernel_size=(1,1),activation='relu',padding='valid',data_format='channels_last',name='block7b_project_conv')(block7b_se_excite)
block7b_project_bn = BatchNormalization(axis=-1,momentum=0.99,epsilon=0.001,name='block7b_project_bn')(block7b_project_conv)
block7b_drop = Dropout(0.2 , name='block7b_drop')(block7b_project_bn)
block7b_add = Add(name='block7b_add')([block7b_drop,block7a_project_bn])


## Sub Block3
block7c_expand_conv = Conv2D(3840,kernel_size=(1,1),data_format='channels_last',padding='valid',activation='relu',strides=(1,1),name='block7c_expand_conv')(block7b_project_bn)
block7c_expand_bn = BatchNormalization(axis=-1,momentum=0.99,epsilon=0.001,name='block7c_expand_bn')(block7c_expand_conv)
block7c_expand_activation = Activation('relu',name='block7c_expand_activation')(block7c_expand_bn)
block7c_dwconv = DepthwiseConv2D(kernel_size=1,activation='relu',padding='valid',data_format='channels_last',name='block7c_dwconv')(block7c_expand_activation)
block7c_bn = BatchNormalization(axis=-1,momentum=0.99,epsilon=0.001,name='block7c_bn')(block7c_dwconv)
block7c_activation = Activation('relu',name='block7c_activation')(block7c_bn)
block7c_se_squeeze = GlobalAveragePooling2D(data_format='channels_last',name='block7c_se_squeeze')(block7c_activation)
block7c_se_reshape = Reshape((1,1,-1),name='block7c_se_reshape')(block7c_se_squeeze)
block7c_se_reduce = Conv2D(160,kernel_size=(1,1),activation='relu',padding='valid',data_format='channels_last',name='block7c_se_reduce')(block7c_se_reshape)
block7c_se_expand = Conv2D(3840,kernel_size=(1,1),activation='relu',padding='valid',data_format='channels_last',name='block7c_se_expand')(block7c_se_reduce)
block7c_se_excite = Multiply(name='block7c_se_excite')([block7c_activation,block7c_se_expand])
block7c_project_conv = Conv2D(640,kernel_size=(1,1),activation='relu',padding='valid',data_format='channels_last',name='block7c_project_conv')(block7c_se_excite)
block7c_project_bn = BatchNormalization(axis=-1,momentum=0.99,epsilon=0.001,name='block7c_project_bn')(block7c_project_conv)
block7c_drop = Dropout(0.2 , name='block7c_drop')(block7c_project_bn)
block7c_add = Add(name='block7b_add')([block7c_drop,block7b_project_bn])

# Final layers

In [14]:
evolved_model = Model(input_layer , block7c_add)

evolved_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        [(None, 216, 384, 1) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 216, 384, 1)  0           input_layer[0][0]                
__________________________________________________________________________________________________
normalization (Normalization)   (None, 216, 384, 1)  3           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 218, 386, 1)  0           normalization[0][0]              
______________________________________________________________________________________________

In [15]:
# B7 = tf.keras.applications.efficientnet.EfficientNetB1(include_top=False, weights=None,input_tensor = input_layer,classifier_activation='softmax')
# B7.summary()